## Application of numpy & matplotlib to mnist data

> Go colab

### Images of hand-written numbers in sample_data folder of colab

In [0]:
ls


In [0]:
ls sample_data

In [0]:
import numpy as np

### train data set
- 20000 images with targets

In [0]:
data = np.loadtxt("sample_data/mnist_train_small.csv", delimiter=",")

In [0]:
data.shape,type(data)

number

ㅣ

1,2,3,......,784,785

.

.

20000

#### Hand-written number images and target level
- data: images of hand-written numbers (28 X 28 pixels)
- target: numbers (0 ~ 9)  

In [0]:
data[:,0]  # target (hand-written numbers)

In [0]:
data[-1,0]

In [0]:
data[-5:,0]

In [0]:
data[:,0].shape  # target of 20,000 hand-written images
# target = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

(20000,) -> numpy 기본 일차원 벡터

(20000,1) -> " 이차원 

In [0]:
target = data[:,0]

In [0]:
target[:5],target[-5:]
# 맨처음 5개, 맨 뒤 5세

In [0]:
ndata = data[:,1:]  # ndata (iamge of hand-written numbers), 784 pixels for each number
print(ndata.shape)
print("784 = {}".format(28*28))
print(type(ndata))

2000 * 784개의 이미지

### 행벡터(1X784)를 이미지 배열 (28X28)로 재구성

data는 전체 data

ndata는 number data

In [0]:
ndata[0,0:]  # first number

In [0]:
ndata[0,0:].shape, target[0]  # 첫번째 숫자는 6   

In [0]:
first_num = ndata[0,0:].reshape(28,28)
last_num = ndata[-1,0:].reshape(28,28)
type(last_num)

In [0]:
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline  #셀에 그래픽이 바로 출력되도록

In [0]:
plt.figure(figsize=(2, 2))  #figure객체 그리기 사이즈(2,2)
plt.imshow(first_num, cmap=mpl.cm.binary) #bone_r)
plt.grid(False)
plt.xticks([])
plt.yticks([])
plt.show()

In [0]:
target[0], target[-1]   # check number 

### Show images of some numbers in train dataset

In [0]:
# function plot_digits
# import numpy as np
# import matplotlib as mpl
def plot_digits(instances, images_per_row=10, **options): # instances -> 내가 그리고 싶은 모든 숫자가 들어감 / images_per_row=10 -> 한줄에 그림 10개씩보여줌
    size = 28
    images_per_row = min(len(instances), images_per_row) # 숫자를 보여줄 때 10개보다 적을 때 한줄에 10개 미만의 그림을 확장해서 보여줌
    images = [instance.reshape(size,size) for instance in instances] # instances에 n * 784개의 숫자가 있음 / images는 50*28*28개의 그림이 있는 리스트(리스트 내포로 만들어서) 리스트에 한 요소에 28*28 개가 총 50개로 이루어진 리스트
    n_rows = (len(instances) - 1) // images_per_row + 1 # 몇줄이 필요한지 계산
    row_images = [] # 한 줄에 들어가는 data갯수 초기화
    n_empty = n_rows * images_per_row - len(instances) # 몇개의 공백을 여백으로 처리할지
    images.append(np.zeros((size, size * n_empty))) # 빈칸에 28*28의 공백으로 채워진다
    for row in range(n_rows): 
        rimages = images[row * images_per_row : (row + 1) * images_per_row] # 보여주려는 숫자 그림들과 공백으로 채워진 그림들이 rimages에 저장
        print(len(rimages)) # rimages가 잘 동작하는지 테스트! (실제 실행시에는 주석처리)
        row_images.append(np.concatenate(rimages, axis=1))  # 10개의 row_images를 채운다 (axis = 1 은 세로?가로?])
        print(len(row_images))
    image_all = np.concatenate(row_images, axis=0) # 10개 묶음으로 만들어서 image_all에 저장
    print(image_all.shape)
    plt.imshow(image_all, cmap = mpl.cm.binary, **options)
    plt.axis("off")

### Draw images of  reshaped ndata

In [0]:
ndata[:34].shape

#### np.r_[], np.c_[], and concatenate()

> [Python의 numpy 라이브러리에 있는 함수들을 이용해서 두 개의 배열(array)을 옆으로 붙이기, 배열을 위 아래로 붙이기](https://rfriend.tistory.com/352)

In [0]:
plt.figure(figsize=(9,9))
example_images = np.r_[ndata[:50]]
plot_digits(ndata[:50], images_per_row=10)
# plot_digits(example_images, images_per_row=10)

plt.show()

print("============================")
print(ndata[:50].shape)
print(example_images.shape)
print(type(example_images))

### 처음 숫자 50개 확인

In [0]:
target[:50].reshape(5,10) # 5행 10열로



---



#### 테스트데이터 샘플을 로드하여 확인해보자.

### Test data set

In [0]:
ls sample_data

In [0]:
tdata = np.loadtxt("sample_data/mnist_test.csv", delimiter=",")

In [0]:
tdata.shape

In [0]:
# target of test data
ty = tdata[:,0]
ty

In [0]:
# test data
tX = tdata[:,1:]

In [0]:
# Plot first 50 digits
plt.figure(figsize=(9,9))
example_images = np.r_[tX[:50]]  # Convert to numpy array
plot_digits(example_images, images_per_row=10)

plt.show()

In [0]:
ty[:50].reshape(5,10)



---



#### train data set를 80% 트래이닝데이터와 20% 검증(validation)데이터로 분리하기
- train data set : ndata  (20000, 784)
- sklearn module 사용

In [0]:
# sklearn module: Python machine learning
from sklearn.model_selection import train_test_split

In [0]:
ndata.shape,ndata.shape[0]

In [0]:
def train_val_split(data, val_pct=0.2):
    size = data.shape[0]
    train, val = train_test_split(data, test_size=val_pct)
    return train, val

In [0]:
train, val = train_val_split(ndata)
train.shape,val.shape

# train_test_split(data, test_size=0.2)

#### 분리된 데이터를 딕셔너리로 관리

In [0]:
dataset = dict()
dataset["train"], dataset["val"] = train_val_split(ndata)

In [0]:
type(dataset["train"]), type(dataset["val"])

In [0]:
dataset["train"].shape, dataset["val"].shape



---



#### train-val-test splitting (80%, 10%, 10%)

In [0]:
def train_val_test_split(data, val_pct=0.1, test_pct=0.1): 
    size = data.shape[0]
    val_pct = (val_pct * size) / (size * (1 - test_pct))
    train_val, test = train_test_split(data, test_size=test_pct)
    train, val = train_test_split(train_val, test_size=val_pct)
    return train, val, test

#### ndata를 60%,20%,20%의 train, val, test 로 분리하고 shape를 확인해보시오.

In [0]:
train,val,test = train_val_test_split(ndata, val_pct=0.2, test_pct=0.2)

In [0]:
train.shape, val.shape, test.shape